<a href="https://colab.research.google.com/github/R0N3ldrt/Thesis/blob/main/spectrum-1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading Enviroment

## Loading Drive and folders

In [13]:
# BLOQUE PARA USAR DESDE COLAB

# Google drive loading as work station for local-usage of the files.
from google.colab import drive
drive.mount('/content/gdrive',force_remount= True)

#-----------------------------------------------------------------------------

Mounted at /content/gdrive


In [14]:
# Lista para cambiar los paths rapido.
workers = ["Ronald", "Local"]

# Cambiar el numero aqui para cambiar los paths.
worker = workers[0]

if worker == "Ronald":
  path= "/content/gdrive/MyDrive/Thesis_Workstation/ANN_dataset"
else: path = os.getcwd()

## Loading Necesary Libraries

In [36]:
# Importing necesary libraries
import os, time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import pickle

# Loading Data and Pre-Procesing

In [90]:
data_path=path+"/Spectrum"

qpsk_path="/QPSK/A.10hops100km/"
qam_path="/16QAM/A.10hops100km"

#file_name="/dataSet_Spectrum_Samples_QPSK_37GHz_LongHaul_output_10x100.xlsx"
file_name="/QPSK_dummy.xlsx"
# Skipping first row since only contain "Frequency" headers
df_qpsk=pd.read_excel(data_path+qpsk_path+file_name, sheet_name = "Sheet1", skiprows=1)

#file_name="/dataSet_Spectrum_Samples_16QAM_75GHz_LongHaul_output_10x100.xlsx"
file_name="/16QAM_dummy.xlsx"
df_qam=pd.read_excel(data_path+qam_path+file_name, sheet_name = "Sheet1", skiprows=1)

print(df_qpsk.head(1))
#print(df_qam.head(3))

#Diff in column 5 to 1,286 0.03125

   sampleid  Channels  Distance_km  power_dBm  #node        -20  -19.96875  \
0         1         1            0         -2      1 -50.396272 -51.195507   

    -19.9375  -19.90625   -19.875  ...   19.71875      19.75   19.78125  \
0 -52.325501 -57.685476 -51.73027  ... -50.701297 -50.425586 -57.242411   

     19.8125  19.84375     19.875   19.90625    19.9375   19.96875         20  
0 -54.978886 -58.98445 -52.289897 -50.007652 -56.562352 -50.661037 -57.857937  

[1 rows x 1286 columns]


In [77]:
#dump_file = dict desireed to be saved
#filename = just the name of the file (file saved on "pickle_folder")

def pickle_out(dump_file, filename):
  filepath = path+"/pickle_folder/"+str(filename)+".pickle"
  pickleout = open(filepath, "wb")
  pickle.dump(dump_file, pickleout)
  pickleout.close()

def pickle_in(filename):
  filepath = path+"/pickle_folder/"+str(filename)+".pickle"
  picklein = open(filepath, "rb")
  loaded_dict = pickle.load(picklein)
  return loaded_dict

In [ ]:
%%----------------------------------------------%%
%%	Computes starting and end line of a file	%%
%%----------------------------------------------%%
%% 20170530

function [X_agg, Y_agg]=aggregator(ChannelData, agg)
% Extracting X:Freq, Y:Power
X=ChannelData(:,1);
Y=ChannelData(:,2);
%     Y = 10 * log10 (Y./10^-3);  % translate to dBm
Y = 10.^(Y/20);

% Initializing aggregated
X_agg=X(1);
Y_agg=Y(1);
cur=1;
while cur<=length(Y)-agg
    X_agg=[X_agg; mean(X(cur:cur+agg-1))];
    Y_agg=[Y_agg; mean(Y(cur:cur+agg-1))];
    cur=cur+agg;
end

Y_agg = 20*log10(Y_agg);

In [106]:
x=1281
prime_num = []
i=2
while i <= 20:
  if (x%i==0):
    prime_num.append(i)
  i+=1
print(prime_num)

[3, 7]


In [183]:
def prime_number_finder(stop_gap, total_num_of_data):
  prime_nums = []
  i=2
  while i <= stop_gap:
    if (total_num_of_data % i==0):
      prime_nums.append(i)
    i+=1
  return prime_nums

def aggreagator_selection(stop_gap=25, total_num_of_data=1281):
  current_freq_offset = 0.03125
  
  # Find the prime numbers where the total_num_of_data is divisible
  prime_nums = prime_number_finder(stop_gap, total_num_of_data)
  
  # Find the offsets of the new aggregators
  new_freq_offsets = []
  for nums in prime_nums:
    freq_offset = current_freq_offset*(nums-1)
    new_freq_offsets.append(freq_offset)

  # Ask the user for the desired aggregator
  print('Please select one of the following prime number to conduct the aggregation {} with frequencies offsets of {}(PHz) respectively. (Current Freq offset is 0.03125 PHz)'.format(prime_nums, new_freq_offsets))
  while True:
    user_selection = input('Prime number to conduct the aggregation? ')
    if int(user_selection) not in prime_nums:
      print('Please select one of the available prime numbers. (Current selection is not avaliable)')
    else:
      break
  return int(user_selection)

def aggreagator(df, stop_gap=25, total_num_of_data=1281):
  agg_num = aggreagator_selection(stop_gap, total_num_of_data)
  agg=0
  loop_cnt=0
  init_column_cnt = 5
  while agg<=total_num_of_data:
    loop_cnt+=1
    agg=(agg_num*loop_cnt)+init_column_cnt
    new_df = df[df.columns[init_column_cnt:agg]]
    new_df['mean_rows'] = new_df.mean(axis = 1)

    break
  return new_df


In [184]:
aggreagator(df_qpsk, stop_gap=100, total_num_of_data=1281)

Please select one of the following prime number to conduct the aggregation [3, 7, 21, 61] with frequencies offsets of [0.0625, 0.1875, 0.625, 1.875](PHz) respectively. (Current Freq offset is 0.03125 PHz)
Prime number to conduct the aggregation? 3


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,-20,-19.96875,-19.9375,mean_rows
0,-50.396272,-51.195507,-52.325501,-51.305760
1,-49.464198,-62.300068,-49.878752,-53.881006
2,-52.100342,-54.825356,-49.933739,-52.286479
3,-61.429210,-54.973444,-45.026184,-53.809613
4,-51.777530,-57.344578,-55.337847,-54.819985
5,-56.348628,-48.323410,-52.791372,-52.487803
6,-59.530214,-46.793059,-54.027619,-53.450297
7,-52.818323,-57.433254,-50.022994,-53.424857
8,-46.037314,-57.909915,-63.732986,-55.893405
9,-49.648529,-50.168222,-54.061412,-51.292721


##Dummy Model

In [74]:
def get_dummy_spectrum(df):
  new_df = df.groupby(['#node']).mean()
  new_df.drop(['sampleid', 'Channels', 'power_dBm'], axis = 1, inplace = True)
  dummy_dict = new_df.to_dict()
  
  return new_df, dummy_dict

In [75]:

#new_df = 
#node	| Distance_km	|-20	      |-19.96875	|-19.9375																						
#1	    | 0.0	        |-52.955056	|-54.126681	|-52.713840	
#2	    | 100.0	      |-51.813386	|-53.194155	|-52.697528
#3	    | 200.0	      |-55.626777	|-54.634329	|-54.702958

#dummy_dict.keys() = dict_keys(['Distance_km', -20, -19.96875, -19.9375, -19.90625, -19.875, -19.84375...
#dummy_dict[-20].values() = dict_values([-52.955056039, -51.813386274, -55.626777088])

In [82]:
dummy_df, dummy_dict = get_dummy_spectrum(df_qpsk)

In [86]:
pickle_out(dummy_dict, "dummy_spectrum_full_QPSK")

In [87]:
pickle_dummy_dict = pickle_in("dummy_spectrum_full_QPSK")

In [88]:
pickle_dummy_dict['Distance_km'].values()

dict_values([0.0, 100.0, 200.0, 300.0, 400.0, 500.0, 600.0, 700.0, 800.0, 900.0, 1000.0])